In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread("the_berry_farms_sunflower_field.jpeg", cv2.IMREAD_REDUCED_COLOR_4)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def LoG(sigma):
    #window size 
    hw = 3*sigma 
    X, Y = np.meshgrid(np.arange(-hw, hw + 1, 1), np.arange(-hw, hw + 1, 1))
    log = 1/(2*np.pi*sigma**2)*(X**2/(sigma**2) + Y**2/(sigma**2) - 2)*np.exp(-(X**2 + Y**2)/(2*sigma**2))
    return log






